# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua

w yaoshangqinghua

wo yaoshangqinghua

woyao shangqinghua

-> DP

# 思路


+ 导入pinyin包 读取数据文件article_9k.txt和comments.txt文件转成拼音形式 存储到变量TOKEN_1_GRAM
+ 统计词典里各个拼音的频数 Counter(TOKEN_1_GRAM) 存储到变量 words_count_1
+ words_count_1 用于计算1-gram语言模型返回word的概率值 和 拼音自动纠错函数
+ 定义pinyin_split函数，
    + 判断n长度的word的概率值，再判断n-1长度的概率值，不断找最优值
    + 将word拆分计算最优结果返回概率值和i索引
+ 定义parse_solution函数将solution结果回溯出来
+ 再将解析的solution结果传到拼音纠错函数correct_sequence_pinyin进行最终校验

In [226]:
# 导入需要用到的包
import pinyin
from collections import Counter, defaultdict
import pandas as pd
import re
from collections import Counter

In [229]:
# 定义函数：调用pinyin包把中文转换成英文
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

# 定义函数：将已经转换为pinyin的文本排除掉数字，返回格式为词列表
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [23]:
TOKEN_1_GRAM = []
for line in (open('./datasets/comments.txt')):   
    TOKEN_1_GRAM += tokens(chinese_to_pinyin(line))

In [24]:
for line in (open('./datasets/article_9k.txt')):
    TOKEN_1_GRAM += tokens(chinese_to_pinyin(line))

In [233]:
# 展示单个词例子
TOKEN_1_GRAM[:10]

['wu', 'jing', 'yi', 'yin', 'dao', 'le', 'nao', 'can', 'de', 'di']

In [234]:
words_count_1 = Counter(TOKEN_1_GRAM)
words_count_1.most_common(10)

[('de', 1195679),
 ('shi', 1173105),
 ('yi', 885644),
 ('ji', 728882),
 ('n', 717796),
 ('zhi', 492174),
 ('guo', 487083),
 ('zhong', 474720),
 ('bu', 442553),
 ('li', 423343)]

In [235]:
# 用words_count_1计算每个拼音出现的概率，这里对不存在的拼音概率设置为了0
def get_probablity_1gram(word):
    # 1-gram模型
    if word in words_count_1:
        
        probability = words_count_1[word] / len(TOKEN_1_GRAM)
#         print("存在",word,probability)
    else:
        # 使用平滑法对不存在的词进行处理
        probability = 0  # 1 / len(TOKEN_1_GRAM)
#         print("不存在",word,probability)

    return  probability

# 定义主函数 

+ 使用动态规划 统计一个words按word[:i]和word[i:]拆分后的概率，返回概率最大的结果
+ 记录拆分概率最大结果的i位置到solution
+ 解析solution获得拼音切分结果

In [237]:
# 拼音切分
def pinyin_split(word):
    if word in memory:
        return memory[word]
    else:
        # 使用 pinyin_split(word[:i]) * pinyin_split(word[i:]概率连乘获得拆分后拼音的概率值
        max_prob, pos = max( [(get_probablity_1gram(word), len(word))] 
                            + [( pinyin_split(word[:i]) * pinyin_split(word[i:]), i) for i in range(1,len(word))])
        # 按pos拆分概率最大，存储到字典solution
        solution[word] = (word[:pos], word[pos:])
        
        memory[word] = max_prob
        
        return max_prob
# 解析words拆分路径
def parse_solution(word):
    left_split, right_split = solution[word]
    
    if right_split == '': return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [254]:
# 初始化变量 
# memory存储各个word的概率值
memory = {}
# solution存储概率最大的拆分结果
solution = {}
# 设定需要拆分的拼音字符串
word = 'woyaoshangqinghua'
pinyin_split(word)

parsed_solution = parse_solution(word)
print("使用语言模型拆分连续拼音的结果为:\n",parsed_solution)

使用语言模型拆分连续拼音的结果为:
 ['wo', 'yao', 'shang', 'qing', 'hua']


# 对解析好的拼音切分结果再进行自动纠错

In [248]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def known(words):
    'Return the pinyin in our data'
#     PINYIN_COUNT修改为words_count_1
    return {w for w in words if w in words_count_1}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)
# 编辑距离为1的词上再加一个编辑距离
def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [249]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    # 找出与它编辑距离为0，1，2的词
    # 注释了[word] 效果等同于known(edits0(word))
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or
                  known(edits2(word)))
    # 返回出现次数最多的candidate
    #     PINYIN_COUNT修改为 words_count_1
    return max(candidates, key=words_count_1.get)

In [250]:
def correct_sequence_pinyin(parsed_solution):
#     把correct函数作用在text里的每一个词上面
    return ' '.join(map(correct, parsed_solution))

In [253]:
result = correct_sequence_pinyin(parsed_solution)

print("经过拼音纠错后最终输出结果为：\n",result)

经过拼音纠错后最终输出结果为：
 wo yao shang qing hua
